In [1]:
#import libraries to use

import pandas as pd
import numpy as np

In [47]:
buyer_data = pd.read_csv('Buyers_RAW.csv',low_memory = False)

In [48]:
buyer_data.head()

,month,year,user_id,currency,count,revenue,Date,USD,Per Purchase
0,6,2015,1154979,GBP,2,37.29,20156,55.94,27.97
1,10,2015,2677378,GBP,1,17,201510,25.50,25.50
2,9,2015,546489,GBP,3,118.5,20159,177.76,59.25
3,8,2015,846185,GBP,4,66.79,20158,100.19,25.05
4,6,2015,2099822,GBP,2,29.98,20156,44.97,22.49


In [49]:
cohort = buyer_data.groupby('user_id',as_index = False)['month'].min()

In [50]:
cohort = pd.DataFrame(cohort)

In [51]:
df = pd.merge(buyer_data,cohort,left_on = 'user_id', right_on = 'user_id')

In [53]:
df.columns = ['month','year','user_id','currency','count','revenue','Date','USD',
               'Per Purchase','cohort']

In [54]:
df.head()

,month,year,user_id,currency,count,revenue,Date,USD,Per Purchase,cohort
0,6,2015,1154979,GBP,2,37.29,20156,55.94,27.97,5
1,5,2015,1154979,GBP,2,10,20155,15.00,7.50,5
2,10,2015,2677378,GBP,1,17,201510,25.50,25.50,10
3,9,2015,546489,GBP,3,118.5,20159,177.76,59.25,8
4,8,2015,546489,GBP,1,17.8,20158,26.70,26.70,8


In [57]:
cohort_analysis = df.groupby(['user_id','cohort','month'],as_index = False)['revenue'].sum()

In [58]:
cohort_analysis

,user_id,cohort,month,revenue
0,6,10,10,71.49
1,19,9,9,22.1
2,30,2,2,20
3,30,2,4,14.32
4,30,2,8,10
5,45,4,4,80
6,58,3,3,8
7,86,4,4,20
8,86,4,5,49.99
9,86,4,10,57
